In [43]:
import pandas as pd
import numpy
from tqdm import tqdm
import requests, zipfile, io
from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np
import re

## READ DATA

In [91]:
dfs = []

for year in tqdm(range(2013, 2019)):
    r = requests.get('http://www.tennis-data.co.uk/{0}/{0}.zip'.format(year))
    z = zipfile.ZipFile(io.BytesIO(r.content))
    df = pd.read_excel(z.open('{0}.xlsx'.format(year)))
    df['year'] = year
    dfs.append(df)

df = pd.concat(dfs)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:18<00:00,  2.74s/it]
C:\Users\Toby\Anaconda2\envs\mne\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [92]:
df.head()

,ATP,AvgL,AvgW,B365L,B365W,Best of,Comment,Court,Date,EXL,...,W1,W2,W3,W4,W5,WPts,WRank,Winner,Wsets,year
0,1,2.78,1.42,3.00,1.36,3,Completed,Outdoor,2012-12-31,2.65,...,6.0,6.0,NaN,NaN,NaN,1215.0,28.0,Mayer F.,2.0,2013
1,1,2.05,1.73,2.20,1.61,3,Completed,Outdoor,2012-12-31,2.00,...,6.0,2.0,6.0,NaN,NaN,927.0,41.0,Nieminen J.,2.0,2013
2,1,3.58,1.28,3.75,1.25,3,Completed,Outdoor,2012-12-31,3.75,...,7.0,6.0,NaN,NaN,NaN,1830.0,19.0,Nishikori K.,2.0,2013
3,1,7.76,1.08,9.00,1.07,3,Completed,Outdoor,2012-12-31,8.00,...,6.0,6.0,NaN,NaN,NaN,1070.0,36.0,Baghdatis M.,2.0,2013
4,1,1.85,1.88,1.80,1.90,3,Completed,Outdoor,2013-01-01,1.87,...,6.0,6.0,NaN,NaN,NaN,897.0,43.0,Istomin D.,2.0,2013


## Compute target variable

In [117]:
df2 = pd.melt(df,id_vars=[c for c in df.columns if not c in ['Winner', 'Loser']], 
              value_vars = ['Winner', 'Loser'],var_name='Outcome', value_name='Player')

df2.loc[df2['Outcome'] == 'Winner', 'OutcomeCode'] = 1
df2.loc[df2['Outcome'] == 'Loser', 'OutcomeCode'] = 0

for column in [i for i in df2.columns if re.match('.+[LW]', i)]:
    if re.match('.+W', column):
        df2.loc[df2['Outcome'] == 'Winner', re.search('.+(?=[WL])', column).group()] = df2.loc[df2['Outcome'] == 'Winner', column]
    elif re.match('.+L', column):
        df2.loc[df2['Outcome'] == 'Loser', re.search('.+(?=[WL])', column).group()] = df2.loc[df2['Outcome'] == 'Loser', column]

df2.head()

,ATP,AvgL,AvgW,B365L,B365W,Best of,Comment,Court,Date,EXL,...,Outcome,Player,OutcomeCode,Avg,B365,EX,LB,Max,PS,SJ
0,1,2.78,1.42,3.00,1.36,3,Completed,Outdoor,2012-12-31,2.65,...,Winner,Mayer F.,1.0,1.42,1.36,1.45,1.44,1.47,1.47,1.44
1,1,2.05,1.73,2.20,1.61,3,Completed,Outdoor,2012-12-31,2.00,...,Winner,Nieminen J.,1.0,1.73,1.61,1.75,1.80,1.80,1.80,1.73
2,1,3.58,1.28,3.75,1.25,3,Completed,Outdoor,2012-12-31,3.75,...,Winner,Nishikori K.,1.0,1.28,1.25,1.25,1.29,1.30,1.30,1.30
3,1,7.76,1.08,9.00,1.07,3,Completed,Outdoor,2012-12-31,8.00,...,Winner,Baghdatis M.,1.0,1.08,1.07,1.06,1.08,1.10,1.08,1.07
4,1,1.85,1.88,1.80,1.90,3,Completed,Outdoor,2013-01-01,1.87,...,Winner,Istomin D.,1.0,1.88,1.90,1.87,1.91,2.05,1.88,1.91


In [21]:
import re
df2 = df2.dropna(subset=[i for i in df2.columns if not re.match('[LW][0-5]', i)])

In [28]:
clf = LogisticRegression()
X = df2[['B365']]
X = scale(X)
y = df2['OutcomeCode']
clf.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [29]:
cross_val_score(clf, X, y)

array([ 0.69215128,  0.70587637,  0.66534472])